Nhiệm vụ 3 – GroupBy & Tổng hợp: 
Mục tiêu : phân nhóm dữ liệu và rút ra các đặc trưng quan trọng. 
- Tính chi phí trung bình theo khoa.
- Tính số lượt khám theo bệnh nhân.
- Tính tỷ lệ BHYT trung bình.
- Xác định khoa có chi phí cao nhất.
- Phân tích liên hệ giữa tuổi và chi phí khám.

In [1]:
import pandas as pd

# ========================================================
# 1. ĐỌC VÀ KẾT NỐI DỮ LIỆU
# ========================================================
base_path = r"D:\OneDrive\Máy tính\18A2_Nhom_4_Case_9\data_clean"

df_p = pd.read_csv(f"{base_path}\\patient_clean.csv")
df_v = pd.read_csv(f"{base_path}\\visit_clean.csv")
df_b = pd.read_csv(f"{base_path}\\billing_clean.csv")

# Merge dữ liệu
df = df_v.merge(df_p, on='patient_id', how='left').merge(df_b, on='visit_id', how='left')


# errors='coerce': Biến mọi giá trị lỗi thành NaN (để không bị dừng chương trình)
df['total_fee'] = pd.to_numeric(
    df['total_fee'].astype(str).str.replace(',', '').str.replace('VND', '').str.strip(),
    errors='coerce'
).fillna(0)

#  ép sang số để tránh lỗi ở câu 3
df['bhyt_clean'] = pd.to_numeric(
    df['insurance_coverage'].astype(str).str.replace('%', ''), 
    errors='coerce'
)
# Chuẩn hóa BHYT (0.8 hoặc 80%)
df.loc[df['bhyt_clean'] > 1, 'bhyt_clean'] = df['bhyt_clean'] / 100
df.loc[df['bhyt_clean'] < 0, 'bhyt_clean'] = float('nan')

print(f"Kiểu dữ liệu cột total_fee hiện tại: {df['total_fee'].dtype}")




Kiểu dữ liệu cột total_fee hiện tại: float64


- Tính chi phí trung bình theo khoa

In [2]:
print("\n--- 1. Chi phí trung bình theo khoa ---")
# Lúc này total_fee đã là số, hàm mean() sẽ hoạt động

avg_fee = df.groupby('department')['total_fee'].mean()
print(avg_fee.apply(lambda x: f"{x:,.0f} VNĐ"))


--- 1. Chi phí trung bình theo khoa ---
department
Ngoai     73,214 VNĐ
Nhi       92,157 VNĐ
Noi      129,762 VNĐ
San      126,744 VNĐ
Name: total_fee, dtype: object


- Tính số lượt khám theo bệnh nhân.

In [3]:
print("\n--- 2. Top 5 Bệnh nhân khám nhiều nhất ---")
print(df['patient_id'].value_counts().head(5))


--- 2. Top 5 Bệnh nhân khám nhiều nhất ---
patient_id
BN139    9
BN112    8
BN138    7
BN119    7
BN109    7
Name: count, dtype: int64


- Tính tỷ lệ BHYT trung bình.

In [4]:
print("\n--- 3. Tỷ lệ BHYT trung bình ---")
print(f"{df['bhyt_clean'].mean() * 100:.2f}%")


--- 3. Tỷ lệ BHYT trung bình ---
55.15%


- Xác định khoa có chi phí cao nhất

In [5]:

print("\n--- 4. Khoa có chi phí trung bình cao nhất ---")
chi_phi_khoa = df.groupby('department')['total_fee'].mean()
ten_khoa = chi_phi_khoa.idxmax()
gia_tri = chi_phi_khoa.max()
print(f"Khoa: {ten_khoa} - Mức giá TB: {gia_tri:,.0f} VNĐ/lượt")


--- 4. Khoa có chi phí trung bình cao nhất ---
Khoa: Noi - Mức giá TB: 129,762 VNĐ/lượt


- Phân tích liên hệ giữa tuổi và chi phí khám.

In [6]:
print("\n--- 5. liên hệ giữa tuổi và chi phí khám ---")
# Phân loại tuổi thành 3 nhóm: Trẻ em, Người lớn, Người già
df['Nhom_Tuoi'] = pd.cut(
    df['age'], 
    bins=[0, 18, 60, 200], 
    labels=['0-18 tuổi', '19-60 tuổi', '> 60 tuổi']
)

# Tính chi phí trung bình theo nhóm tuổi
xu_huong = df.groupby('Nhom_Tuoi', observed=True)['total_fee'].mean()
print(xu_huong.apply(lambda x: f"{x:,.0f} VNĐ"))


--- 5. liên hệ giữa tuổi và chi phí khám ---
Nhom_Tuoi
0-18 tuổi      60,000 VNĐ
19-60 tuổi    110,714 VNĐ
Name: total_fee, dtype: object
